In [12]:
import cv2
import urllib.request
import numpy as np

# Define the URL of the MJPEG stream
url = "http://192.168.1.110:81/stream"  # Replace with your ESP32 camera's stream URL

def capture_face():
    # Open the MJPEG stream using OpenCV
    img_resp = urllib.request.urlopen(url)  # Open the stream URL

    # Read the MJPEG stream frame by frame
    byte_data = b''  # Initialize a byte buffer for the stream
    while True:
        byte_data += img_resp.read(1024)  # Read data in chunks
        a = byte_data.find(b'\xff\xd8')  # JPEG start marker
        b = byte_data.find(b'\xff\xd9')  # JPEG end marker

        if a != -1 and b != -1:
            jpg = byte_data[a:b+2]  # Extract the JPEG frame
            byte_data = byte_data[b+2:]  # Keep the remaining data for next frame

            # Convert the byte data to an image
            img_np = np.frombuffer(jpg, dtype=np.uint8)
            img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)

            # Check if image is captured correctly
            if img is not None:
                # Display the image using OpenCV
                cv2.imshow("ESP32 Camera Stream", img)

                # Optionally, use a face detection model (e.g., OpenCV's Haar cascades)
                face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Detect faces
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Draw rectangle around the face

                # Show the image with detected faces
                cv2.imshow("Face Detection", img)

            # Wait for a key press to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Release resources
    cv2.destroyAllWindows()

if __name__ == "__main__":
    capture_face()


KeyboardInterrupt: 